In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [7]:
df = pd.read_csv('../dados/bank-full.csv', sep=';')

In [3]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
#pipeline de transformaçõo das colunas
colunas_numericas = ['age', 'balance', 'duration']
colunas_numericas_categoricas = ['day', 'campaign', 'pdays', 'previous']
colunas_categoricas = ['job', 'marital', 'education', 'default', 'housing',
       'month', 'poutcome']

In [7]:
#Transformador de colunas. Salvar como template. Criar texto com explicação.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [8]:
preprocessador = ColumnTransformer(transformers=[
    ('num', numeric_transformer, colunas_numericas),
    ('cat_numero', categorical_numerical_transformer, colunas_numericas_categoricas),
    ('categorica', categorical_transformer, colunas_categoricas)
])

In [10]:
x = df[['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome']]
y = df['y']

In [11]:
treino, teste, resposta_treino, resposta_teste = train_test_split(x, y, random_state = 42, test_size = 0.2)

In [12]:
#método para treinar com todas as categorias, pra evitar a questão do risco de amostragem
train_objs_num = len(treino)
dataset = pd.concat(objs=[treino,teste], axis=0)
dataset_tratado = preprocessador.fit_transform(dataset)
treino_tratado = dataset_tratado[:train_objs_num]
teste_tratado = dataset_tratado[train_objs_num:]

In [13]:
#Aqui vai a aplicação do modelo de machine learning